#Workshop 3

##Workshop definition:

As you are becoming an expert on understand systems, it is the time to create an expert system.

Also, as you shoud know, you professor love movies a lot. So, you want to help him to choose next movies to watch. In this sense, you will take information for next movies from a popular website called IMDb where there is different information about movies and opinions.

However, you do not want to spend money to get movies information using a webAPI. But, as you are a good computer engineer, you know there is a technique called web scrapping, and you could get information using HTML from an website, and put this information into a DataFrame just to make easier data processing.

In this case, you are gonna use request package to get HTML code from IMDb site, beautiful soup package to make web scrapping, and pandas to have a dataframe to handle data.

In [ ]:
# neccesary libraries
!pip install requests
!pip install beautifulsoup4
!pip install pandas

#1. Get Movies Data
In the link IMDb Upcoming Relases there are next movies with some related data as genre or star actors. So, using html tags, you could take information and save it into python variables. At the end, here you want to have a DataFrame with movies information.

In [ ]:
# import libraries
import requests
from bs4 import BeautifulSoup

def get_web_html(url: str) -> BeautifulSoup:
    """
    This method gets the HTML from a website using scrapping.

    Args:
        url: URL to scrape.

    Returns:
        A BeautifulSoup object with the HTML.
    """
    # headers to avoid 403 error, cos' IMDb blocks requests from bots
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }

    # get HTML and save it in a variable
    response = requests.get(url, headers=headers)
    return BeautifulSoup(response.text, "html.parser")

In [ ]:
# import libraries
from bs4 import BeautifulSoup


def get_title_year(movie_data: BeautifulSoup) -> tuple:
    """
    This methog gets the title, year, and link of a movie data using scrapping.

    Args:
        movie_data: BeautifulSoup object with movie data.

    Returns:
        A tuple with title, year, and link.
    """
    # process main title of subdivission
    base_url = "https://www.imdb.com"
    ref_data = movie_data.find("a")
    title = ref_data.text.split("(")[0].strip()
    year = ref_data.text.split("(")[1].replace(")", "")
    link = base_url + ref_data["href"]
    return title, year, link


def get_genre_actors(movie_data: BeautifulSoup) -> list:
    """
    This method gets genre and actors from a movie data using scrapping.

    Args:
        movie_data: BeautifulSoup object with movie data.

    Returns:
        A list with genre and actors.
    """
    # process ul tags to get genre and actors
    data = []
    ul_tags = movie_data.find_all("ul")
    for ul_tag in ul_tags:
        temp = []
        for li_tag in ul_tag.find_all("li"):
            temp.append(li_tag.find("span").get_text())
        data.append(", ".join(temp))
    return data

In [10]:
# import libraries
from bs4 import BeautifulSoup
import pandas as pd


def generate_dataframe(url: str) -> pd.DataFrame:
    """
    This method generates a DataFrame with movie data from IMDb.

    Args:
        url: URL to scrape.

    Returns:
        A DataFrame with movie data.
    """
    # movies datastructure definition
    imdb = get_web_html(url)
    movies = []
    movies_metadata = ["Title", "Year", "Genre", "Actors"]

    # process HTML using scrapping going to each division with the class ipc-metadata-list-summary-item__tc
    movies_html = imdb.find_all("div", class_="ipc-metadata-list-summary-item__tc")
    for movie in movies_html:
        # get each movie data into a clean html structure
        movie_data = BeautifulSoup(str(movie), "html.parser")

        # get movie data
        title, year, link = get_title_year(movie_data)
        data = get_genre_actors(movie_data)

        # create a dictionary to have a nice data movie structure
        movie_clean_data = {
            "Title": title,
            "Year": year,
            "Genre": data[0],
            "Actors": data[1] if len(data) > 1 else "",
        }
        # create a list of dictionaries to create a DataFrame
        movies.append(movie_clean_data)

    # create movies dataframe
    return pd.DataFrame(movies, columns=movies_metadata)

# ================================ MAIN =================================== #
# url to scrape
url = "https://www.imdb.com/calendar/?ref_=rlm&region=US&type=MOVIE"
movies_df = generate_dataframe(url)
print(movies_df.size)

720


#2. Time to build a Decision Tree
In order to create an expert system, you must define a decision tree. It means, a sequence of conditionals in order to get a result, it is like to build a flowchart composed for a lot of conditionals.

In this part, you must to go to any site like draw.io and create a flowchart about how you think is the best decision process: what questions do you want to make?

Remember, make the right questions is a pretty important step at any taks you want to do.

As follows is the algorithm used in the expert system

#2.1 Your proposal
In this section you must write your system analysis, and justify the questions selection and order. Also, define system components, system sensibility, anything you consider important to solve this problem.

Into this analisys that I've made about the database that we got directly from the webpage of IMB, I checked directly inside of the page the informaiton that the program were getting
With this I noticed that normally the program generate a dataframes of movies of 720 movioes in total, all of them with the directly information of the full title of the movie in the spanish languaje, also it get the year of release and the common genres in where the movie have been clasified, added to this the dataframe get the information of all the principal actors, with this information I decided to order the questions to filtrate the program, these are the questions and the order that I'll be using onto my program
1. Which movie gender do you want to see?
This will help me in order to filtrate between the different movie genders that we have, reducing the amount of movies that the system can recommend, for example into the first try that I've made in order to filter the movies, I try to check the movies that have the "Drama" genre on them, this have reduced the amount of movies from 702 to 316, almost the half of the movies where filtrated for not having the selected genre on them

In [13]:
filtered_movies_genre = movies_df[movies_df['Genre'].str.contains('Drama', na=False)]
print(filtered_movies_genre.size)

316


2.which release year do you want to see in your movie?  
After I filtrate the amount of movies based on the genre, the best option that I got to continue filtering them were the year in where the movie were released, this because as we know the year of a movie can decide the amount of special efects that can be on it, the production that were made, or also even the plot of the movie, for example the movies of the "Terror" genre on the years previous of 2000 were made as slasher movies, while the released after that year, are more enfocated into Psychological suspense
But when I were making the process of the code, I noticed that the program only have the option to get movies between 2023 and 2025, that's why I needed to make an conditional to check that the answer to the movie will be one of the available years (with my example of bellow this reduced the filter from 316 to 60 movies

In [23]:
filtered_movies_year = filtered_movies_genre[filtered_movies_genre['Year'].str.contains('2025', na=False)]
print(filtered_movies_year.size)
print(filtered_movies_year.head(10))

60
                                              Title  Year  \
152                                       Mickey 17  2025   
154                                   Game of Power  2025   
156                Bridget Jones: Mad About the Boy  2025   
158                         Verona's Romeo & Juliet  2025   
159                             The Unbreakable Boy  2025   
160  Untitled Ryan Coogler / Michael B. Jordan Film  2025   
162                                      Snow White  2025   
164                           The Woman in the Yard  2025   
167                        Bonhoeffer: Holy Traitor  2025   
168                             The Perfect Destiny  2025   

                           Genre  \
152     Adventure, Drama, Sci-Fi   
154        Crime, Drama, Mystery   
156       Comedy, Drama, Romance   
158      Drama, Musical, Romance   
159                        Drama   
160     Drama, Fantasy, Thriller   
162     Adventure, Drama, Family   
164      Drama, Horror, Thriller   
1

3. Do you have a prefered actor that you want to see into the movie?
 Which one?

As I have already filtered the 3/4 part of the movies I want to reduce the amount to the minimal amount of movies possible, what I've done with the quesiton of the actor is found a selected parameter that will no completely repeated on the movies, something that will be specific just for a certainly amount of them, on my example I checked with the actor Robert Pattinson, and I reduced the amount of movies from 60 to 4

In [26]:
filtered_movies_actors = filtered_movies_year[filtered_movies_year['Actors'].str.contains('Robert Pattinson', na=False)]
print(filtered_movies_actors.size)
print(filtered_movies_actors.head(4))

         Title  Year                     Genre  \
152  Mickey 17  2025  Adventure, Drama, Sci-Fi   

                                                Actors  
152  Robert Pattinson, Mark Ruffalo, Toni Collette,...  


#3. Put your expert system at work
Now it is the time to write code in order to create your first and classical expert system. Here it is important to use conditionals, and based on conditionals apply filters in pandas dataframe.

In [37]:
genre = input("Which movie gender do you want to see?(Action, Comedy, Drama, History. etc)")
print(f"You have put: {genre}")
filtered_movies_genre = movies_df[movies_df['Genre'].str.contains(genre, na=False)]
if filtered_movies_genre.size == 0:
  print("No movies found")
else:
  print(filtered_movies_genre.size)
  print(filtered_movies_genre.head(5))

year = input("which release year do you want to see in your movie?(2023, 2024, 2025)")
print(f"You have put: {year}")
filtered_movies_year = filtered_movies_genre[filtered_movies_genre['Year'].str.contains(year, na=False)]
if filtered_movies_year.size == 0:
  print("No movies found")

else:
  print(filtered_movies_year.size)
  print(filtered_movies_year.head(5))

actor = input("Do you have a prefered actor that you want to see into the movie?(Yes, No. etc)")
print(f"You have put: {actor}")
filtered_movies_actors = filtered_movies_year[filtered_movies_year['Actors'].str.contains(actor, na=False)]
if filtered_movies_actors.size == 0:
  print("No movies found")
else:
  print("\n These are the recommended movies based on your decision for the genre, the year and the actor")
  print(filtered_movies_actors.head(5))





Which movie gender do you want to see?(Action, Comedy, Drama, History. etc)Drama
You have put: Drama
316
                                         Title  Year  \
2                                        Sight  2023   
3  Kidnapped: The Abduction of Edgardo Mortara  2023   
7                          In a Violent Nature  2024   
8                                         Ezra  2023   
9                          The Dead Don't Hurt  2023   

                       Genre  \
2  Biography, Drama, History   
3             Drama, History   
7    Drama, Horror, Thriller   
8              Comedy, Drama   
9             Drama, Western   

                                              Actors  
2  Terry Chen, Greg Kinnear, Natasha Mumba, Fionn...  
3  Paolo Pierobon, Fausto Russo Alesi, Barbara Ro...  
7  Ry Barrett, Andrea Pavlovic, Cameron Love, Ree...  
8  Robert De Niro, Vera Farmiga, Tony Goldwyn, Ro...  
9  Vicky Krieps, Viggo Mortensen, Solly McLeod, G...  
which release year do you want to s